# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | classes.ipynb
 |-data\Orcus - Classes.xlsx
 |-output
```

and will take data\Orcus - Classes.xlsx and create a markdown file output\classes.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Classes.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

,Name,Completed,Tradition,Role,HP at 1st Level,HP at Higher Levels,Defense Bonuses,Recoveries per Long Rest,Armor Proficiencies,Weapon Proficiencies,...,7 - Desc,Notes,Key Ability,Class Disciplines - List,Talents,Talents Text,Dualclass,Art,Art Link,Art Zoom
0,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,a,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,,,,
1,Commander,1,Martial,Leader,12,5,+2 Fortitude,7,"Cloth, leather, hide, chainmail, scale; light ...","Simple melee, martial melee; simple ranged",...,,,Charisma,"Angel's Trumpet, Golden Lion",,Choose one at character creation. \n\n* **Stor...,You belong to the Commander class (along with ...,Rook by Jared von Hindman,pics\rook.jpg,100.0
2,Crusader,1,Divine,Leader,12,5,+2 Will,7,"Cloth, leather, hide, chainmail","Simple melee, simple ranged",...,,,Wisdom,Angel's Trumpet,,"Choose one kit that begins with ""Worships …"". ...",You belong to the Crusader class (along with a...,,,
3,Guard,1,Martial,Defender,15,6,+2 Fortitude,9,"Cloth, leather, hide, chainmail, scale; light ...","Simple melee, martial melee, simple ranged, ma...",...,,,Strength,"Art of War, Juggernautical",,Choose one at character creation. \n\n* **Grea...,You belong to the Guard class (along with any ...,,,
4,Jester,1,Martial,Controller,10,4,+2 Will,6,"Cloth, leather","Simple melee, simple ranged",...,,,Charisma,Last Laugh,,Choose one at character creation.\n\n* **Clown...,You belong to the Jester class (along with any...,,,


In [3]:
#cell 2, examine 5 random classes

data.sample(5)

,Name,Completed,Tradition,Role,HP at 1st Level,HP at Higher Levels,Defense Bonuses,Recoveries per Long Rest,Armor Proficiencies,Weapon Proficiencies,...,7 - Desc,Notes,Key Ability,Class Disciplines - List,Talents,Talents Text,Dualclass,Art,Art Link,Art Zoom
15,Wilder,,Martial,Striker/Defender,15,6,"+1 Fortitude, +1 Will",8,"Cloth, leather, hide","Simple melee, martial melee, simple ranged, ma...",...,,,,,,,,,,
5,Mageblade,1,Arcane,Defender,15,6,"+1 AC, +2 Will",8,"Cloth, leather","Simple melee, martial melee; simple ranged",...,,,Intelligence,"Elemental Flux, Veiled Moon",,Choose one at character creation. \n\n* **Illu...,You belong to the Mageblade class (along with ...,,,
2,Crusader,1,Divine,Leader,12,5,+2 Will,7,"Cloth, leather, hide, chainmail","Simple melee, simple ranged",...,,,Wisdom,Angel's Trumpet,,"Choose one kit that begins with ""Worships …"". ...",You belong to the Crusader class (along with a...,,,
3,Guard,1,Martial,Defender,15,6,+2 Fortitude,9,"Cloth, leather, hide, chainmail, scale; light ...","Simple melee, martial melee, simple ranged, ma...",...,,,Strength,"Art of War, Juggernautical",,Choose one at character creation. \n\n* **Grea...,You belong to the Guard class (along with any ...,,,
16,Captain,,Martial,Leader,12,5,+2 Will,7,"Cloth, leather, hide, chainmail, scale; light ...","Simple melee, martial melee; simple ranged",...,,,,,,,,,,


In [4]:
#cell 3
#we define a function which takes in a row of this power dataframe and returns an md string

def class_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
   # if row_dict.get('Name') == 'Chapter':
   #     md = '# {Chapter}  '.format(**row_dict)
   #     if row['Description']:
   #         md += "\n{Description}  ".format(**row_dict)
   #     return md

    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
        
    #start with name. md is the string we'll be outputing
    md = """\n# {Name}  """.format(**row_dict)
    
    #this is a complex chunk
    md += """\n**{Tradition} {Role}**  """.format(**row_dict)
    
    if row['Desc']:
        md += "\n{Desc}  ".format(**row_dict)

    
    md += """\n## Stats  """.format(**row_dict)
    
    md += """\n**Hit Points at 1st Level:** {HP at 1st Level}.  """.format(**row_dict)
    
    md += """\n**Hit Points at Higher Levels:** {HP at Higher Levels}.  """.format(**row_dict)
    
    md += """\n**Recoveries per Long Rest:** {Recoveries per Long Rest}.  """.format(**row_dict)
    
    md += """\n**Defenses:** {Defense Bonuses}.  """.format(**row_dict)
    
    md += """\n## Proficiency and Training  """.format(**row_dict)
    
    if row['Armor Proficiencies']:
        md += "\n**Armor Proficiencies:** {Armor Proficiencies}.  ".format(**row_dict)

    if row['Weapon Proficiencies']:
        md += "\n**Weapon Proficiencies:** {Weapon Proficiencies}.  ".format(**row_dict)

    if row['Focus Proficiencies']:
        md += "\n**Focus Proficiencies:** {Focus Proficiencies}.  ".format(**row_dict)
        
    if row['Trained Skills']:
        md += "\n**Trained Skills:** You are trained in {Trained Skills} and {Other Skills} other skills from your class skills.  ".format(**row_dict)
    else:
        md += "\n**Trained Skills:** You are trained in {Other Skills} skills from your class skills.  ".format(**row_dict)
                      
    md += """\n**Class Skills:**  {Class Skills}.  """.format(**row_dict)

    md += """\n## Features  """.format(**row_dict)
    

    if row['Feature 1']:
        md += "\n### {Feature 1}  \n{1 - Desc}  """.format(**row_dict)
    
    if row['Feature 2']:
        md += "\n### {Feature 2}  \n{2 - Desc}  """.format(**row_dict)
            
    if row['Feature 3']:
        md += "\n### {Feature 3}  \n{3 - Desc}  """.format(**row_dict)
    
    if row['Feature 4']:
        md += "\n### {Feature 4}  \n{4 - Desc}  """.format(**row_dict)
        
    if row['Feature 5']:
        md += "\n### {Feature 5}  \n{5 - Desc}  """.format(**row_dict)
    
    if row['Feature 6']:
        md += "\n### {Feature 6}  \n{6 - Desc}  """.format(**row_dict)
        
    if row['Feature 7']:
        md += "\n### {Feature 7}  \n{7 - Desc}  """.format(**row_dict)

    if row['Talents Text']:
        md += "\n### Talents  \n{Talents Text}  """.format(**row_dict)
            
    if row['Class Disciplines - List']:
        md += "\n### Powers  """.format(**row_dict)
        md += "\nYou can choose powers from your class disciplines. When you do so, you can replace any reference to the discipline's key ability with {Key Ability} instead. You can replace any reference to the discipline's secondary ability with the secondary ability linked to your talent.  """.format(**row_dict)
        md += "\n\n**Class Disciplines:** {Class Disciplines - List}.  """.format(**row_dict)

    if row['Dualclass']:
        md += "\n### Dualclass  \nIf a character takes the Dualclass Recruit feat and selects the {Name}, it has the following benefit:  """.format(**row_dict)
        md += "\n\n**Benefit:** {Dualclass}  """.format(**row_dict)        

    if row['Art']:
        md += "\n<figure><img src='{Art Link}' alt='{Art}' style='zoom: {Art Zoom}%;' /><figcaption>{Art}</figcaption></figure>  """.format(**row_dict)

        
    #finally we remove the empty tags
    return md.replace('+-','-')

In [5]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = class_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [6]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')

In [7]:
#cell 6
# now we have to save to disk

fname =  'Orcus Classes.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))